In [ ]:
!git clone https://github.com/ClementPinard/FlowNetPytorch.git

In [ ]:
pip install -r FlowNetPytorch/requirements.txt

In [2]:
import cv2
import numpy as np
from FlowNetPytorch.models.FlowNetS import FlowNetS  
from FlowNetPytorch.models.FlowNetS import flownets

In [3]:
import torch
checkpoint_path='flownets_EPE1.951.pth.tar'
net = flownets()
state_dict = torch.load(checkpoint_path,map_location=torch.device('cpu'))
net.load_state_dict(state_dict['state_dict'])
net.eval()

FlowNetS(
  (conv1): Sequential(
    (0): Conv2d(6, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): LeakyReLU(negative_slope=0.1, inplace=True)
  )
  (conv2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (1): LeakyReLU(negative_slope=0.1, inplace=True)
  )
  (conv3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (1): LeakyReLU(negative_slope=0.1, inplace=True)
  )
  (conv3_1): Sequential(
    (0): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.1, inplace=True)
  )
  (conv4): Sequential(
    (0): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.1, inplace=True)
  )
  (conv4_1): Sequential(
    (0): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.1, inplace=True)
  )
  (conv5): Sequential(
    (0): Conv2d(512, 512,

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import os
import torch.nn.functional as F

In [5]:
import torchvision.transforms as transforms
from FlowNetPytorch import flow_transforms
from FlowNetPytorch.util import flow2rgb

input_transform = transforms.Compose([
        flow_transforms.ArrayToTensor(),
        transforms.Normalize(mean=[0,0,0], std=[255,255,255]),
        transforms.Normalize(mean=[0.411,0.432,0.45], std=[1,1,1])
    ])



In [9]:
from path import Path

In [22]:
class ImageDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        
        data_dir=Path(data_dir)
       
        self.frames = []
        img_ext=['png', 'jpg', 'bmp', 'ppm']
        
        for ext in img_ext:
            test_files = data_dir.files('*1.{}'.format(ext))
            for file in test_files:
                img_pair = file.parent / (file.stem[:-1] + '2.{}'.format(ext))
                if img_pair.isfile():
                    self.frames.append([file, img_pair])
        

                          
        
                       
                   
                    
                   
    def __len__(self):
        return len(self.frames)
   
    def __getitem__(self, index):
        xx=self.frames[index]
        frame1 = cv2.imread(xx[0])
        frame1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2RGB)
        
        frame2 = cv2.imread(xx[1])
        frame2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2RGB)
        
       
        
       
        if self.transform is not None:
            frame1 = self.transform(frame1)
            frame2 = self.transform(frame2)
            

       
        return frame1,frame2

In [25]:
datset=ImageDataset('inference',input_transform)

In [26]:
train_loader = DataLoader(datset, shuffle=True)

In [27]:
for i, data in enumerate(train_loader, 0):
    img1,img2=data
    input_var = torch.cat((img1,img2),1)
    
    output = net(input_var)
    
    h,w=img1.size()[-2:]

    output=F.interpolate(output, size=(h, w), mode='bicubic', align_corners=False)
    
    for k in range(output.shape[0]):
        rgb_flow = flow2rgb(output[k],1)
        to_save = (rgb_flow * 255).astype(np.uint8).transpose(1,2,0)
        cv2.imwrite('rgb'+str(i)+'.jpg',to_save)
    

In [28]:
def get_point(image,t,x,y):
    c,h,w=image.shape
    if t>=0 and t<c and x>=0 and x<h and y>=0 and y<w:
        return image[t][x][y]
    return 0

In [29]:
def grad_x(image,t,x,y):
    x1=0
    x2=0
    for i in range(2):
        for j in range(2):
            x1+=get_point(image,t+i,x+1,y+j)
            x2+=get_point(image,t+i,x,y+j)
            
    ans=(x1-x2)/4
    return ans


def grad_y(image,t,x,y):
    x1=0
    x2=0
    
    for i in range(2):
        for j in range(2):
            x1+=get_point(image,t+i,x+j,y+1)
            x2+=get_point(image,t+i,x+j,y)
    
    ans=(x1-x2)/4
    return ans


def grad_t(image,t,x,y):
    x1=0
    x2=0
    for i in range(2):
        for j in range(2):
            x1+=get_point(image,t+1,x+i,y+j)
            x2+=get_point(image,t,x+i,y+j)
  
    ans=(x1-x2)/4
    return ans

In [30]:


def optical_flow(img1, img2, W):
    
    # opticalFlow calculates the displacements in X and Y directions i.e., (u,v)
    # given two consecutive images varying with time

    n, m = img1.shape
    arr = torch.zeros((2, n, m))

    w = W // 2

    image = torch.zeros((2, n + 2 * w, m + 2 * w))

    gx = torch.zeros((n + 2 * w, m + 2 * w))
    gy = torch.zeros((n + 2 * w, m + 2 * w))

    gt = torch.zeros((n + 2 * w, m + 2 * w))

    for i in range(w, n + w):
        for j in range(w, m + w):
#             print(i,j)
            image[0][i][j] = img1[i - w][j - w]
            image[1][i][j] = img2[i - w][j - w]

    for i in range(n + 2 * w):
        for j in range(m + 2 * w):
#             print(i,j)
            gx[i][j] = grad_x(image, 0, i, j)  # calculating gradient
            gy[i][j] = grad_y(image, 0, i, j)
            gt[i][j] = grad_t(image, 0, i, j)

    for i in range(w, n + w):
        for j in range(w, m + w):

            window_gx = gx[i - w:i + w + 1, j - w:j + w + 1]
            window_gy = gy[i - w:i + w + 1, j - w:j + w + 1]

            A = torch.vstack((window_gx.flatten(), window_gy.flatten())).T
            b = -gt[i - w:i + w + 1, j - w:j + w + 1].flatten()

            u = torch.matmul(torch.pinverse(A), b)

            arr[0][i - w][j - w] = u[0]
            arr[1][i - w][j - w] = u[1]

    return arr


In [31]:
datset2=ImageDataset('inference')
train_loader2 = DataLoader(datset2, shuffle=True)

In [41]:
for i, data in enumerate(train_loader2, 0):
    img1,img2=data
    k,h,w,c=img1.shape

    
    for j in range(k):
        image1=img1[j].numpy()
        image2=img2[j].numpy()
        image1=cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
        image2=cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
        image1=torch.from_numpy(image1)
        image2=torch.from_numpy(image2)
        output=optical_flow(image1,image2,3)
        
        
        rgb_flow = flow2rgb(output,1)
        to_save = (rgb_flow * 255).astype(np.uint8).transpose(1,2,0)
        c1=i*k+j
        cv2.imwrite('rgb_lucas'+str(c1)+'.jpg',to_save)
    
    
    
    
    
    